### Retrieval and Query Agent

Goals: The end goal is to create a smart agent, that can retrieve and query data from a set of documents and spreadsheets, and reason about the data. 

1. Create a retrieval and query agent that can answer questions about a set of documents.
2. Use the Gemini API to generate embeddings for the documents and queries.
3. Use the ChromaDB to store the embeddings and query them.
4. Use the Gemini API to generate a response to a query using the embeddings.
5. Use the Gemini API to generate a response to a query from spreadsheets.
6. Use the Gemini API to RAG.
7. Reasoning with the Gemini API from a spreadsheet.



Steps:

- Can I store spreadsheet data in a vector database?
- How can I store documents from different sources?
- How much my Agent is able to answer questions about the data after training?


In [46]:
import google.generativeai as genai
from IPython.display import Markdown

genai.configure(api_key="GOOGLE_API_KEY")

In [47]:
for model in genai.list_models():
    if "embedContent" in model.supported_generation_methods:
        print(model.name)

models/embedding-001
models/text-embedding-004


### Create a baseline model

In [59]:
# Initialize the Gemini model
SYSTEM_MESSAGE = '''Você é um agente especializado em dar informações sobre a cidade Chácara em Minas Gerais.
Você é um agente que responde sobre questões históricas, culturais e geografia. Além de dar informações relevantes que você tenha na sua base de dados que receber de nós, como horário de ônibus.
Você não responderá questões que não sejam sobre chácara e caso alguém pergunte, você responderá educadamente que não pode dar estas informações.
'''

baseline_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    system_instruction=SYSTEM_MESSAGE
    )

# Start a chat
chat = baseline_model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
print(chat.history)

[parts {
  text: "Hello"
}
role: "user"
, parts {
  text: "Great to meet you. What would you like to know?"
}
role: "model"
]


### Test the Model - Section:

Examples of queries:

In [5]:
# Exemplos de perguntas:

# Qual me fale sobre a história de chácara.
#

In [49]:
from ipywidgets import widgets
from IPython.display import display, clear_output

# Create widgets
text_input = widgets.Text(
    value='',
    placeholder='Type your message here...',
    description='',
    layout={'width': '500px'}
)
send_button = widgets.Button(description="Send")
output_area = widgets.Output()

# Chat history display
def on_send_clicked(b):
    with output_area:
        print(chat.history)
        clear_output()
        response = chat.send_message(text_input.value)
        print(f"You: {text_input.value}")
        print(f"Assistant: {response.text}\n")
        text_input.value = ''  # Clear input

send_button.on_click(on_send_clicked)

# Display interface
display(text_input, send_button, output_area)

Text(value='', layout=Layout(width='500px'), placeholder='Type your message here...')

Button(description='Send', style=ButtonStyle())

Output()

### Data

In [50]:
import pandas as pd

data = pd.read_csv("data/chácara_knowledge.csv", 
                   encoding='latin1',
                   sep=',',           # Specify the delimiter
                   quotechar='"',     # Specify quote character
                   escapechar='\\',   # Handle escaped characters
                   on_bad_lines='warn'  # or 'skip' to skip problematic rows
                   )

In [51]:
data.head(10)

,TÃ³pico,ConteÃºdo
0,HistÃ³ria,"ChÃ¡cara surgiu como povoado, em 1863, na faze..."
1,InformaÃ§Ãµes,Aqui estÃ¡ o horÃ¡rio de Ã´nibus de Juiz de Fo...
2,Geografia,AspÃ©ctos Naturais\n O ambiente natural de ChÃ...
3,Geografia,CaracterÃ­sticas\n O municÃ­pio de ChÃ¡cara/M....
4,HistÃ³ria,"Bandeira e BrasÃ£o\n A Lei Municipal nÂº 188, ..."
5,HistÃ³ria,TÃ­tulo: Te amo cidade querida!\n \n Autor: Se...


In [52]:
# Create a list to store all documents
documents = []

# Iterate through the DataFrame and append content to documents list
for _, row in data.iterrows():
    documents.append(row['ConteÃºdo'])  # Using the content column

# Verify the documents (optional)
print(f"Number of documents stored: {len(documents)}")

Number of documents stored: 6


In [53]:
documents

['ChÃ¡cara surgiu como povoado, em 1863, na fazenda da Cachoeira, de propriedade de Severino Dias Tostes. Ali foi passada no dia 10 de janeiro daquele ano, a escritura de doaÃ§Ã£o de cinco alqueires de terras, no valor de cento e cinqÃ¼enta contos de rÃ©is, feita por Iria Maria da Silva para a construÃ§Ã£o do PatrimÃ´nio da Capela de SÃ£o SebastiÃ£o. Serviu como escrivÃ£o no ato, o Senhor Francisco Delgado Pinto, assinando a rodo da doadora que nÃ£o sabia escrever, o cidadÃ£o AntÃ´nio JosÃ© Duque e, como testemunhas, os senhores AntÃ´nio Henrique Barbosa, Joaquim AntÃ´nio da Silva, AntÃ´nio JosÃ© Fonseca Sobrinho, Manoel AntÃ´nio da Fonseca, Romualdo Vieira e AcÃ¡cio da Costa Pinto. Ao redor da capela desenvolveu-se o povoado que, poucos anos depois, isto Ã©, em 1870, foi elevado Ã\xa0 categoria de distrito de paz. E trÃªs anos mais tarde, foi criado o distrito policial. A lei nÂº 3276, de 30 de outubro de 1884, elevou o distrito Ã\xa0 condiÃ§Ã£o de freguesia. A divisÃ£o administrativa

In [54]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

class GeminiEmbeddingFunction(EmbeddingFunction): # This is a custom embedding function that uses the Gemini API to generate embeddings.
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings: # This is the function that generates the embeddings.
        if self.document_mode:
            embedding_task = "retrieval_document"  # This is the task that will be used to generate the embeddings.
        else:
            embedding_task = "retrieval_query"  # This is the task that will be used to generate the embeddings.

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        
        return response["embedding"]


In [55]:
import chromadb

DB_NAME = "characadb"
embed_fn = GeminiEmbeddingFunction() # This is the embedding function that will be used to generate the embeddings.

embed_fn.document_mode = True # This is the mode that will be used to generate the embeddings.

chroma_client = chromadb.Client() # This is the client that will be used to interact with the ChromaDB.
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn) # This is the collection that will be used to store the embeddings.

db.add(documents=documents, ids=[str(i) for i in range(len(documents))]) # This is the function that will be used to add the documents to the collection.

In [56]:
db.count()

6

### Pesquise sobre Informações de Chácara:

In [57]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Me fale sobre a história de Chácara"

result = db.query(query_texts=[query], n_results=1) # This is the function that will be used to search the ChromaDB.
[[passage]] = result["documents"] # This is the passage that will be used to answer the question.

Markdown(passage) # This is the function that will be used to display the passage.

ChÃ¡cara surgiu como povoado, em 1863, na fazenda da Cachoeira, de propriedade de Severino Dias Tostes. Ali foi passada no dia 10 de janeiro daquele ano, a escritura de doaÃ§Ã£o de cinco alqueires de terras, no valor de cento e cinqÃ¼enta contos de rÃ©is, feita por Iria Maria da Silva para a construÃ§Ã£o do PatrimÃ´nio da Capela de SÃ£o SebastiÃ£o. Serviu como escrivÃ£o no ato, o Senhor Francisco Delgado Pinto, assinando a rodo da doadora que nÃ£o sabia escrever, o cidadÃ£o AntÃ´nio JosÃ© Duque e, como testemunhas, os senhores AntÃ´nio Henrique Barbosa, Joaquim AntÃ´nio da Silva, AntÃ´nio JosÃ© Fonseca Sobrinho, Manoel AntÃ´nio da Fonseca, Romualdo Vieira e AcÃ¡cio da Costa Pinto. Ao redor da capela desenvolveu-se o povoado que, poucos anos depois, isto Ã©, em 1870, foi elevado Ã  categoria de distrito de paz. E trÃªs anos mais tarde, foi criado o distrito policial. A lei nÂº 3276, de 30 de outubro de 1884, elevou o distrito Ã  condiÃ§Ã£o de freguesia. A divisÃ£o administrativa do Estado, em 1962, criou o municÃ­pio de ChÃ¡cara, que foi oficialmente instalada em 1Âº de marÃ§o de 1963. Anualmente em janeiro a cidade promove festa em homenagem ao padroeiro SÃ£o SebastiÃ£o, alÃ©m do carnaval, um dos maiores eventos culturais da cidade, mantendo a tradiÃ§Ã£o dos concurso dos mascarados, trazendo para a cidade nesta Ã©poca turistas de vÃ¡rias regiÃµes. E a ExposiÃ§Ã£o AgropecuÃ¡ria e o Torneio Leiteiro, geralmente no final do mÃªs de agosto, e reconhecido como um dos melhores rodeio da regiÃ£o. A histÃ³ria de ChÃ¡cara registra a presenÃ§a de importantes personalidades como CÃ¢ndido Teixeira Tostes, BarÃ£o de Catas Altas, Severino Dias Tostes, Teodorico Ribeiro de Assis, Dr. JosÃ© ProcÃ³pio Teixeira, Padre Leopoldo Caglianoni (primeiro vigÃ¡rio), Dr. JoÃ£o D`Avila, Cel Agenor Augusto Sampaio, CustÃ³dio Augusto de Resende e outros.

In [58]:
# Initialize the Gemini model
SYSTEM_MESSAGE = '''Você é um agente especializado em dar informações sobre a cidade Chácara em Minas Gerais.
Você é um agente que responde sobre questões históricas, culturais e geografia. Além de dar informações relevantes que você tenha na sua base de dados que receber de nós, como horário de ônibus.
Você não responderá questões que não sejam sobre chácara e caso alguém pergunte, você responderá educadamente que não pode dar estas informações.
'''

# model_id = f"chacara-model-{random.randint(10000, 99999)}" # model id

chacara_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    system_instruction=SYSTEM_MESSAGE
    )

# Start a chat
chat = chacara_model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
print(chat.history)

[parts {
  text: "Hello"
}
role: "user"
, parts {
  text: "Great to meet you. What would you like to know?"
}
role: "model"
]


In [62]:
from ipywidgets import widgets
from IPython.display import display, clear_output

# Create widgets
text_input = widgets.Text(
    value='',
    placeholder='Type your message here...',
    description='',
    layout={'width': '500px'}
)
send_button = widgets.Button(description="Send")
output_area = widgets.Output()

# Chat history display
def on_send_clicked(b):
    with output_area:
        clear_output()
        
        # First, search for relevant context using ChromaDB
        embed_fn.document_mode = False
        
        # Na busca do ChromaDB
        result = db.query(
            query_texts=[text_input.value],  # O texto da pergunta do usuário
            n_results=2                      # Número de documentos mais relevantes a retornar
        )
        
        context = "\n".join(result["documents"][0])  # Join relevant passages
        
        # Combine user question with context
        augmented_query = f"""
        Contexto: {context} # Documentos relevantes encontrados
        
        Pergunta do usuário: {text_input.value} # Pergunta original
        
        Por favor, responda à pergunta usando as informações do contexto acima.
        """
        
        # Send augmented query to chat
        response = chat.send_message(augmented_query)
        
        # Display interaction
        print(f"You: {text_input.value}")
        print(f"Assistant: {response.text}\n")
        text_input.value = ''  # Clear input

send_button.on_click(on_send_clicked)

# Display interface
display(text_input, send_button, output_area)

Text(value='', layout=Layout(width='500px'), placeholder='Type your message here...')

Button(description='Send', style=ButtonStyle())

Output()

In [64]:
print(chat.history)

[parts {
  text: "Hello"
}
role: "user"
, parts {
  text: "Great to meet you. What would you like to know?"
}
role: "model"
, parts {
  text: "\n        Contexto: CaracterÃ­sticas\n O municÃ­pio de ChÃ¡cara/M.G. localiza-se no sudeste da Zona da Mata Mineira, na MicroregiÃ£o de Juiz de Fora, sendo o centro urbano localizado em um vale recortado pelo cÃ³rrego SÃ£o SebastiÃ£o, cuja denominaÃ§Ã£o nos remete ao padroeiro deste municÃ­pio. A localizaÃ§Ã£o geogrÃ¡fica da sede municipal, a 21Â° 40â 19â S e 43Â° 13â 19â O, a aproximadamente 15 quilÃ´metros de Juiz de Fora, Ã© privilegiada em relaÃ§Ã£o a importantes capitais como o Rio de Janeiro e Belo Horizonte, alÃ©m de outros centros urbanos como Coronel Pacheco/M.G. e GoianÃ¡/M.G.. Com Ã¡rea de 152,88 quilÃ´metros quadrados e altitudes que variam entre 639 metros na foz do CÃ³rrego Ãgua Limpa e 1023 metros na nascente do RibeirÃ£o das Rosas, ChÃ¡cara tem como municÃ­pios limÃ­trofes: Goiana ao norte, Coronel Pacheco a noroeste, Ju

## Important Notes:

Aqui estamos usando um vector dabase para fazer pesquisas de informações relevantes. No entanto, se eu quizesse passar as informações de texto direto para o conhecimento da generative ia, como ficaria? Comparando as duas formas, quais são os impactos?

Ótima pergunta! Vou mostrar as duas abordagens e comparar seus impactos:
Abordagem Direta (Passando todo o conhecimento no system prompt):

### Abordagem Direta (Passando todo o conhecimento no system prompt):

In [63]:
# Concatenar todos os documentos
all_knowledge = "\n".join(documents)

# System message com conhecimento embutido
SYSTEM_MESSAGE_WITH_KNOWLEDGE = f'''Você é um agente especializado em dar informações sobre a cidade Chácara em Minas Gerais.
Você é um agente que responde sobre questões históricas, culturais e geografia.

Aqui está seu conhecimento sobre Chácara:
{all_knowledge}

Você não responderá questões que não sejam sobre chácara e caso alguém pergunte, você responderá educadamente que não pode dar estas informações.
'''

direct_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash-latest",
    system_instruction=SYSTEM_MESSAGE_WITH_KNOWLEDGE
)

# Função de chat simplificada
def on_send_clicked_direct(b):
    with output_area:
        clear_output()
        response = chat.send_message(text_input.value)
        print(f"You: {text_input.value}")
        print(f"Assistant: {response.text}\n")
        text_input.value = ''
        

### Abordagem com Vector Database (RAG) - como está no seu código atual:


In [ ]:
def on_send_clicked_rag(b):
    with output_area:
        clear_output()
        result = db.query(
            query_texts=[text_input.value],
            n_results=2
        )
        context = "\n".join(result["documents"][0])
        
        augmented_query = f"""
        Contexto: {context}
        Pergunta do usuário: {text_input.value}
        """
        response = chat.send_message(augmented_query)
        print(f"You: {text_input.value}")
        print(f"Assistant: {response.text}\n")

Comparação dos Impactos:
1. Limitações de Contexto:

Abordagem Direta:
- ❌ Limitada pelo tamanho máximo do contexto do modelo
- ❌ Todo o conhecimento compete pelo mesmo espaço de contexto

Vector DB/RAG:
- ✅ Pode lidar com grandes volumes de dados
- ✅ Recupera apenas informações relevantes

2. Precisão das Respostas:

Abordagem Direta:
- ✅ Acesso a todo o conhecimento de uma vez
- ❌ Pode misturar informações irrelevantes

Vector DB/RAG:
- ✅ Respostas mais focadas e precisas
- ✅ Melhor em encontrar informações específicas

3. Desempenho:

Abordagem Direta:
- ✅ Mais rápida (sem busca)
- ❌ Pode ser mais lenta para processar todo o contexto

Vector DB/RAG:
- ❌ Overhead da busca vetorial
- ✅ Processamento mais eficiente do contexto relevante

4. Manutenção e Atualização:

Abordagem Direta:
- ❌ Precisa reinicializar o modelo para atualizar conhecimento
- ❌ Mais difícil de gerenciar

Vector DB/RAG:
- ✅ Fácil adicionar/remover documentos
- ✅ Atualizações dinâmicas

5. Escalabilidade:

Abordagem Direta:
- ❌ Não escala bem com mais dados
- ❌ Limitada pelo tamanho do contexto

Vector DB/RAG:
- ✅ Altamente escalável
- ✅ Pode crescer conforme necessário

Recomendação:

Para bases de conhecimento pequenas (< 2000 tokens): Abordagem Direta pode ser mais simples
Para bases maiores ou que precisam de atualizações frequentes: RAG é mais adequado
No seu caso específico, como você está trabalhando com documentos sobre Chácara e pode precisar atualizar ou adicionar mais informações no futuro, a abordagem RAG que você está usando é provavelmente a melhor escolha.